In [38]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Importing data
col_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "type"]
data = pd.read_csv("iris.csv", skiprows=1, header=None, names=col_names)
data.head(10)

# Node class
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # Decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # Leaf node
        self.value = value

# DecisionTreeClassifier class
class DecisionTreeClassifier:
    def __init__(self, min_sample_splits, max_depth):
        # Initialize the root of the tree
        self.root = None

        # Stopping conditions
        self.min_sample_splits = min_sample_splits
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        '''Recursive function to build tree'''

        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        # Split until the stopping conditions are met
        if num_samples >= self.min_sample_splits and curr_depth <= self.max_depth:
            # Find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # Check if information gain is positive
            if best_split["info_gain"] > 0:
                # Left recursion
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                # Right recursion
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                # Return decision node
                return Node(best_split['feature_index'], best_split['threshold'], left_subtree, right_subtree, best_split['info_gain'])

        # Compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # Return leaf node
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        # Dictionary to store the best split
        best_split = {}
        max_info_gain = -float('inf')

        # Loop over all features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            # Loop over all feature values
            for threshold in possible_thresholds:
                # Get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # Check if children are non-empty
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # Compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # Update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        # Return best split
        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)

        if mode == "gini":
            return self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            return self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))

    def entropy(self, y):
        entropy = 0
        unique_values, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        for prob in probabilities:
            entropy -= prob * np.log2(prob)
        return entropy

    def gini_index(self, y):
        gini = 1
        unique_values, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        for prob in probabilities:
            gini -= prob ** 2
        return gini

    def calculate_leaf_value(self, y):
        y = list(y)
        return max(set(y), key=y.count)

    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)
        else:
            print(f"X_{tree.feature_index} <= {tree.threshold} ? (Info gain: {tree.info_gain})")
            print(f"{indent}left:", end="")
            self.print_tree(tree.left, indent + indent)
            print(f"{indent}right:", end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        ''' Function to train the tree '''
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        ''' Function to predict new dataset '''
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    def make_prediction(self, x, tree):
        ''' Function to predict a single data point '''
        if tree.value is not None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

# Splitting dataset
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=41)

# Training and printing the tree
classifier = DecisionTreeClassifier(min_sample_splits=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.print_tree()


X_2 <= 1.9 ? (Info gain: 0.33741385372714494)
 left:Setosa
 right:X_3 <= 1.5 ? (Info gain: 0.427106638180289)
  left:X_2 <= 4.9 ? (Info gain: 0.051246537396121776)
    left:Versicolor
    right:Virginica
  right:X_2 <= 5.0 ? (Info gain: 0.019631171921475288)
    left:X_1 <= 2.8 ? (Info gain: 0.20833333333333331)
        left:Virginica
        right:Versicolor
    right:Virginica


In [39]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.9333333333333333